## env

In [45]:
!nvidia-smi -L

GPU 0: Tesla V100-SXM2-16GB (UUID: GPU-abcd01d2-2774-d8d2-723e-214718439f8f)


In [46]:
# !pip install --upgrade -q pip tensorflow-datasets mne torch torchvision
%pip install mne torch torchvision tensorflow-datasets tensorboard

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [47]:
# !rm -rf ~/.cache/pip

In [48]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [49]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
import sys
sys.path.append('/content/drive/MyDrive/MT_ML_Decoding/CNN/load/code')
from load_data import load_MEG_dataset
import os
import torch
import torch.nn as nn
import torch.utils.data as Data
import torchvision
import matplotlib.pyplot as plt
import numpy as np

In [50]:
device = torch.device('cuda' if  torch.cuda.is_available() else 'cpu')

In [51]:
device

device(type='cuda')

# CNN

## Load data

In [ ]:
X_train, y_train = load_MEG_dataset([str(i).zfill(3) for i in range(1,2)], mode = 'concatenate', output_format='numpy', training=True, batch_size=500)
X_test, y_test = load_MEG_dataset([str(i).zfill(3) for i in range(1,2)], mode = 'concatenate', output_format='numpy', training=False, batch_size=500)

Loading subject 001
Download complete
Data loaded
Subject 001 complete
--------------------------------------
Loading subject 001
Data loaded
Subject 001 complete
--------------------------------------


In [ ]:
X_train.shape

(675, 272, 11)

In [ ]:
X_train = X_train[:, None, ...]
X_test = X_test[:, None, ...]

In [ ]:
X_train.shape

(675, 1, 272, 11)

In [ ]:
X_train=np.repeat(X_train,17*3*11,axis=1)
X_test=np.repeat(X_test,17*3*11,axis=1)

In [ ]:
X_train.shape # 16*17*11=44*68

(675, 561, 272, 11)

In [ ]:
for n, i in enumerate(np.unique(y_train)):
  y_train[y_train == i] = n
  y_test[y_test == i] = n

In [ ]:
import numpy as np
X_train = torch.from_numpy(X_train.astype(np.float32))
# y_train = torch.from_numpy(y_train.astype(np.float32)).type(torch.LongTensor)
y_train = torch.from_numpy(y_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
# y_test = torch.from_numpy(y_test.astype(np.float32)).type(torch.LongTensor)
y_test = torch.from_numpy(y_test.astype(np.float32))

In [ ]:
X_train = X_train.reshape(X_train.shape[0],3,4*17*11,17*4*11)
X_test = X_test.reshape(X_test.shape[0],3,4*17*11,17*4*11)

In [ ]:
X_train.shape

torch.Size([675, 3, 748, 748])

In [ ]:
X_test.shape

torch.Size([225, 3, 748, 748])

In [ ]:
y_train.shape

torch.Size([675])

## Models

### params

In [ ]:
EPOCH=500
LR=1e-3
BATCH_SIZE = 1
model_choice = 'Unet'

In [ ]:
train = Data.TensorDataset(X_train, y_train)
test = Data.TensorDataset(X_test, y_test)

train_loader = Data.DataLoader(train, batch_size = BATCH_SIZE, shuffle = False)
test_loader = Data.DataLoader(test, batch_size = BATCH_SIZE, shuffle = False)

In [ ]:
n_classes = len(np.unique(y_train))
print(n_classes)

14


### CNN models

In [ ]:
if model_choice == 'test1':
  class CNN(nn.Module):
    def __init__(self):
      # super(CNN, self)._init_()
      super(CNN, self).__init__()
      self.conv1 = nn.Sequential(
          nn.Conv2d(
              in_channels=3,
              out_channels=32,
              kernel_size=3,
              stride=1,
              padding=1,
          ),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2,stride=2),
      )
      self.conv2 = nn.Sequential(
          nn.Conv2d(32,64,3,1,1),
          nn.ReLU(),
          nn.MaxPool2d (2,2),
      )
      # self.fc = nn.Linear(64*7*7,128)
      self.fc = nn.Linear(11968, 100)
      self.out = nn.Linear(100,n_classes)
      self.softmax = nn.Softmax()

    def forward(self,x):
      x=self.conv1(x)
      x=self.conv2(x)
      # x=x.view(x.size(0),-1)
      x = torch.flatten(x, 1) # flatten all dimensioxns except batch
      x=self.fc(x)
      x=self.out(x)      
      # output=self.out(x)
      # return output, x
      x=self.softmax(x)
      return x

if model_choice == 'test2':
  class CNN(nn.Module):
    def __init__(self):
      super(CNN, self).__init__()
      self.conv1 = nn.Sequential(
          nn.Conv2d(3,32,3,1,1),
          nn.ReLU(),
          nn.MaxPool2d(2),
      )
      self.conv2 = nn.Sequential(
          nn.Conv2d(32,16,3,1,1),
          nn.ReLU(),
          nn.MaxPool2d(2),
      )
      self.fc1 = nn.Linear(17*16*11, 120)
      self.fc2 = nn.Linear(120, 84)
      self.out = nn.Linear(84, n_classes)
      self.softmax =nn.Softmax()

    def forward(self,x):
      x=self.conv1(x)
      x=self.conv2(x)
      x=x.view(x.size(0),-1)
      # print('x1:',x.shape)
      # x = torch.flatten(x, 1) # flatten all dimensions except batch
      # print('x2:',x.shape)
      x=self.fc1(x)
      x=self.fc2(x)
      x=self.out(x)
      x=self.softmax(x)
      return x

if model_choice == 'alexnet':
  model = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=False)
  fc_features = model.fc.in_features
  model.fc = nn.Linear(fc_features, 9)
  model.eval()

if model_choice == 'resnet18':
 class CNN(nn.Module):
    def __init__(self):
      super(CNN, self).__init__()
      self.conv1 = nn.Sequential(
          nn.Conv2d(1,3,3,1,1),
          torch.hub.load('facebookresearch/semi-supervised-ImageNet1K-models', 'resnet18_swsl',pretrained=False))
      
if model_choice == 'INCEPTION_V3':
  model = torch.hub.load('pytorch/vision:v0.10.0', 'inception_v3', pretrained=False)
  model.eval()  

if model_choice =='YOLO':
  model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=False)
  model.eval()  


In [ ]:
model_choice

'Unet'

In [ ]:
if model_choice == 'test1' or model_choice == 'test2':
  cnn = CNN().to(device)
  print('test:\n',cnn)
if model_choice == 'alexnet':
  cnn = model.to(device)
  print('alexnet:\n', cnn)
if model_choice == 'resnet18':
  resn = model.to(device)
  resn = resneXt.eval().to(device)
  print('resnet18:\n',resn)
if model_choice == 'INCEPTION_V3':
  cnn = model.cuda()
  print(cnn)
if model_choice =='YOLO':
  cnn = model.cuda()
  print(cnn)

### Unet

In [ ]:
UNET = torch.hub.load('mateuszbuda/brain-segmentation-pytorch', 'unet',
    in_channels=3, out_channels=1, init_features=32, pretrained=False)


Using cache found in /root/.cache/torch/hub/mateuszbuda_brain-segmentation-pytorch_master


In [ ]:
unet = UNET.cuda()
print(unet)

UNet(
  (encoder1): Sequential(
    (enc1conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (enc1norm1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (enc1relu1): ReLU(inplace=True)
    (enc1conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (enc1norm2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (enc1relu2): ReLU(inplace=True)
  )
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (encoder2): Sequential(
    (enc2conv1): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (enc2norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (enc2relu1): ReLU(inplace=True)
    (enc2conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (enc2norm2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, tra

## Train

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
optimizer = torch.optim.Adam(unet.parameters(), lr =LR) #momentum =?
loss_func=nn.CrossEntropyLoss()

In [ ]:
import torch.nn.functional as F

for epoch in range(EPOCH):  # loop over the dataset multiple times

  running_loss = 0.0
  for i, data in enumerate(train_loader, 0):
    # get the inputs; data is a list of [inputs, labels]
    
    inputs, labels = data
    labels = labels.type(torch.LongTensor)
    # print('inputs:',inputs.shape)
    # print('labels:',labels.shape)
    # labels = F.one_hot(labels.to(torch.int64), 
    #                    n_classes)
    inputs, labels = inputs.to(device), labels.to(device)
    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = unet(inputs)
    # print(outputs.shape)
    # predicted,_ = torch.max(outputs, 1)
    # print(predicted)
    # print(labels)
    loss = loss_func(outputs, labels)
    # print((outputs, 1)[0],labels)
    # loss = loss_func((outputs, 1)[0],labels)
    # loss = loss_func(outputs,labels.to(torch.float))
    # # loss = loss_func(outputs,labels)
    # print(loss.item())

    loss.backward()
    optimizer.step()
    # print('outputs:',outputs.shape)
    # print(loss.item())

    # #accuracy
    # correct += (outputs == labels).float().sum()
    # print('correct: ', correct)
    # accuracy = 100 * correct / len(np.unique(y_train))
    # # accuracy = outputs/

    # print statistics
    running_loss += loss.item()
    # print(i)
    if i % 50 == 0:    # print every 5000 mini-batches
    # if 0 == 0:    # print every 5000 mini-batches
        # print(f'[{epoch + 1}, {i + 1:5d}] loss: {loss.item() / 2000:.3f}')
        print(f'[{epoch + 1}, {i + 1}] loss: {running_loss}')

        running_loss = 0.0 

print('Finished Training')

# PATH = './cnn.pth'
# torch.save(model.state_dict(), PATH)

RuntimeError: ignored

In [ ]:
torch.cuda.memory_summary(device=None, abbreviated=False)


'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 1            |        cudaMalloc retries: 1         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |   14329 MB |   14329 MB |   14333 MB |    3283 KB |\n|       from large pool |   14327 MB |   14327 MB |   14330 MB |    3283 KB |\n|       from small pool |       2 MB |       2 MB |       2 MB |       0 KB |\n|---------------------------------------------------------------------------|\n| Active memory         |   14329 MB |   14329 MB |   14333 MB |    3283 KB |\n|       from large pool |   14327 MB |   14327 MB |

In [ ]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(n_classes)]
    n_class_samples = [0 for i in range(n_classes)]
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = cnn(inputs)
        optimizer.step()          
        # max returns (value ,index)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        print(n_samples)
        n_correct += (predicted == labels).sum().item()
        print(n_correct)
        # for i in range(24):
        for i in range(n_classes):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label.to(torch.int64)] += 1
            n_class_samples[label.to(torch.int64)] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    # classes = ('1','2','3','4','5','6','7','8')
    # for i in range(10):
    #     acc = 100.0 * n_class_correct[i] / n_class_samples[i]
    #     print(f'Accuracy of {classes[i]}: {acc} %')

In [ ]:
outputs.shape

In [ ]:
outputs[0]

In [ ]:
(outputs, 1)[0].shape

In [ ]:
predicted

In [ ]:
test = torch.max(outputs, 1)
print(test)

In [ ]:
predicted

In [ ]:
labels

In [ ]:
labels = labels.to(torch.int64)

In [ ]:
labels

# LSTM RNN

## Load

In [ ]:
from torch.autograd import Variable
from torch.cuda import amp
import torch.nn.functional as F
import math
%env CUBLAS_WORKSPACE_CONFIG=:16:8

env: CUBLAS_WORKSPACE_CONFIG=:16:8


In [ ]:
Split = 0.90

In [ ]:
X_train, y_train = load_MEG_dataset([str(i).zfill(3) for i in range(1,2)], mode = 'concatenate', output_format='numpy',shuffle = False, training=True, train_test_split=Split)#, batch_size=500)
X_test, y_test = load_MEG_dataset([str(i).zfill(3) for i in range(1,2)], mode = 'concatenate', output_format='numpy',shuffle = False, training=False, train_test_split=Split)#, batch_size=500)

Loading subject 001
Data loaded
Subject 001 complete
--------------------------------------
Loading subject 001
Data loaded
Subject 001 complete
--------------------------------------


In [ ]:
# X_train, X_test = (X_train-X_train.mean())/X_train.std(), (X_test-X_test.mean())/X_test.std() #standardization 

In [ ]:
y_train = (y_train / 2).astype(int) - 1
y_test = (y_test / 2).astype(int) - 1

In [ ]:
X_train_tensors = torch.Tensor(X_train)
X_test_tensors = torch.Tensor(X_test)
# y_train_tensors = torch.from_numpy(y_train.astype(np.float32)) # for mean-squared error
# y_test_tensors = torch.from_numpy(y_test.astype(np.float32))
y_train_tensors = torch.from_numpy(y_train.astype(int)) # for cross entropy loss
y_test_tensors = torch.from_numpy(y_test.astype(int))

In [ ]:
#reshaping to rows, timestamps, features
X_train_tensors_final = torch.permute(X_train_tensors, (0,2,1))
X_test_tensors_final = torch.permute(X_test_tensors, (0,2,1)) 
# X_train_tensors_final = X_train_tensors
# X_test_tensors_final = X_test_tensors

In [ ]:
X_test_tensors_final.shape

torch.Size([90, 130, 272])

One hot

In [ ]:
def onehot(batches, n_classes, y):
  yn = torch.zeros(batches, n_classes)
  for i in range(batches):
    x = [0 for j in range(batches)]
    x[i] = y[i]/2-1                     #ex. [12]-> [5]
    yn[i][int(x[i])]+= 1                  #[000010000]
  return yn

In [ ]:
y_train_tensors_onehot = onehot(int(y_train_tensors.size(0)), len(np.unique(y_train)), y_train_tensors)
y_test_tensors_onehot = onehot(int(y_test_tensors.size(0)), len(np.unique(y_train)), y_test_tensors)
print(y_train_tensors_onehot.shape)

torch.Size([810, 14])


In [ ]:
X_train_tensors_final=X_train_tensors_final.cuda()
X_test_tensors_final=X_test_tensors_final.cuda()
y_train_tensors=y_train_tensors.cuda()
y_test_tensors=y_test_tensors.cuda()

In [ ]:
BATCH_SIZE=10
train = Data.TensorDataset(X_train_tensors_final, y_train_tensors)
test = Data.TensorDataset(X_test_tensors_final, y_test_tensors)
train_loader = Data.DataLoader(train, batch_size = BATCH_SIZE, shuffle = True)
test_loader = Data.DataLoader(test, batch_size = BATCH_SIZE, shuffle = True)
#onehot loader
train_onehot = Data.TensorDataset(X_train_tensors_final, y_train_tensors_onehot)
test_onehot = Data.TensorDataset(X_test_tensors_final, y_test_tensors_onehot)
train_loader_onehot = Data.DataLoader(train_onehot, batch_size = BATCH_SIZE, shuffle = False)
test_loader_onehot = Data.DataLoader(test_onehot, batch_size = BATCH_SIZE, shuffle = False)

In [ ]:
X_train_tensors_final.shape

torch.Size([810, 130, 272])

In [ ]:
y_train_tensors_onehot.shape

torch.Size([810, 14])

## Parms

In [ ]:
num_epochs = 10000
learning_rate = 0.00001 #0.001 lr

input_size = 272 #number of features
hidden_size = 10 #number of features in hidden state
num_layers = 1 #number of stacked lstm layers

num_classes = len(np.unique(y_train)) #number of output classes 

In [ ]:
CRED    = '\33[31m'
CGREEN  = '\33[32m'
CYELLOW = '\33[33m'
CBLUE   = '\33[34m'

## model

In [ ]:
class LSTM1(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length):
        super(LSTM1, self).__init__()
        self.num_classes = num_classes #number of classes
        self.num_layers = num_layers #number of layers
        self.input_size = input_size #input size
        self.hidden_size = hidden_size #hidden state
        self.seq_length = seq_length #sequence length

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                          num_layers=num_layers, batch_first=True) #lstm
        self.fc_1 =  nn.Linear(hidden_size, 128) #fully connected 1
        self.fc = nn.Linear(128, num_classes) #fully connected last layer

        self.relu = nn.ReLU()
    
    def forward(self,x):
        h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)).to(device) #hidden state
        c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)).to(device) #internal state
        # Propagate input through LSTM
        output, (hn, cn) = self.lstm(x, (h_0, c_0)) #lstm with input, hidden, and internal state
        hn = hn.view(-1, self.hidden_size) #reshaping the data for Dense layer next
        out = self.relu(hn)
        out = self.fc_1(out) #first Dense
        out = self.relu(out) #relu
        out = self.fc(out) #Final Output
        return out

In [ ]:
lstm1 = LSTM1(num_classes, input_size, hidden_size, num_layers, X_train_tensors_final.shape[1]).to(device) #our lstm class 
print(lstm1)

LSTM1(
  (lstm): LSTM(272, 10, batch_first=True)
  (fc_1): Linear(in_features=10, out_features=128, bias=True)
  (fc): Linear(in_features=128, out_features=14, bias=True)
  (relu): ReLU()
)


In [ ]:
# criterion = torch.nn.MSELoss()    # mean-squared error for regression
# criterion = torch.nn.CosineEmbeddingLoss()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(lstm1.parameters(), lr=learning_rate) 

In [ ]:
# for epoch in range(num_epochs):
#   outputs = lstm1(X_train_tensors_final) #forward pass
#   optimizer.zero_grad() #caluclate the gradient, manually setting to 0
 
#   # obtain the loss function
#   loss = criterion(outputs, y_train_tensors)
 
#   loss.backward() #calculates the loss of the loss function
 
#   optimizer.step() #improve from loss, i.e backprop
#   if epoch % 100 == 0:
#     print("Epoch: %d, loss: %1.5f" % (epoch, loss.item())) 

In [ ]:
  # for i, data in enumerate(train_loader, 0):
  #   # get the inputs; data is a list of [inputs, labels]
  #   inputs, labels = data
  #   print()

In [ ]:
# from torch.utils.tensorboard import SummaryWriter
# if lstm1 != None:
#   tb = SummaryWriter()
#   inputs, labels = next(iter(train_loader))
#   grid = torchvision.utils.make_grid(inputs)
#   # tb.add_image("inputs", grid)
#   tb.add_graph(lstm1, inputs)
#   tb.close()
# summary_writer = tensorboard.SummaryWriter(f'./models/test')

In [ ]:
def GETcorrectnumber(loader, printcolor):
  with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(num_classes)]
    n_class_samples = [0 for i in range(num_classes)]
    for inputs, labels in loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = lstm1(inputs)
        optimizer.step()          
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        # n_correct += (predicted == labels).sum().item()
        for k in range(predicted.shape[0]):
          if predicted[k]==labels[k]:
            n_correct +=1
        # for i in range(num_classes): # accuracy for each class
        #     label = labels[i]
        #     pred = predicted[i]
        #     if (label == pred):
        #         n_class_correct[i] += 1
        #     n_class_samples[i] += 1
    acc = 100.0 * n_correct / n_samples
    print(printcolor+f'[{epoch + 1}] t accuracy： {acc}%'+printcolor)
  return acc

In [ ]:
def GETcorrectnumberonehot(loader, printcolor):
  with torch.no_grad():
    n_correct = 0
    n_samples = 0
    accuracy = 0
    n_class_correct = [0 for i in range(num_classes)]
    n_class_samples = [0 for i in range(num_classes)]
    for inputs, labels in loader:
      inputs = inputs.to(device)
      labels = labels.to(device)
      outputs = lstm1(inputs)
      optimizer.step()          
      _, predicted = torch.max(outputs, 1)
      n_samples += labels.size(1)
      n_correct += (predicted[1] == labels[1]).sum().item()
      for i in range(num_classes):
          label = labels[i]
          pred = predicted[i]
          if (label == pred):
              n_class_correct[i] += 1
          n_class_samples[i] += 1
    
  #   print(printcolor+f'[{epoch + 1}] train accuracy： {accuracy}%'+printcolor)
  

In [ ]:
# def classification_metric(loader):
#     for inputs, labels in loader:
#       inputs = inputs.to(device)
#       labels = labels.to(device)
#       outputs = lstm1(inputs)
#       optimizer.step()          
#       _, predicted = torch.max(outputs, 1)
#       pred_labels = predicted
#       true_labels = labels

#       assert 1 >= pred_labels.all() >= 0
#       assert 1 >= true_labels.all() >= 0

#       # True Positive (TP): we predict a label of 1 (positive), and the true label is 1.
#       TP = torch.sum((pred_labels == 1) & ((true_labels == 1)))

#       # True Negative (TN): we predict a label of 0 (negative), and the true label is 0.
#       TN = torch.sum((pred_labels == 0) & (true_labels == 0))

#       # False Positive (FP): we predict a label of 1 (positive), but the true label is 0.
#       FP = torch.sum((pred_labels == 1) & (true_labels == 0))

#       # False Negative (FN): we predict a label of 0 (negative), but the true label is 1.
#       FN = torch.sum((pred_labels == 0) & (true_labels == 1))
#       print(TP, TN, FP, FN)
#       return (TP, TN, FP, FN)

## Train

In [ ]:
from torch.utils.tensorboard import SummaryWriter
summary_writer = SummaryWriter(f'./models/test')
train_loss = []
valid_loss = []
lstm1.train()
for epoch in range(num_epochs):  # loop over the dataset multiple times
  t_loss = 0
  for i, data in enumerate(train_loader, 0):
    # get the inputs; data is a list of [inputs, labels]
    inputs, labels = data
    # print(labels)
    inputs, labels = inputs.to(device), labels.to(device)
    # zero the parameter gradients
    optimizer.zero_grad()
    # forward + backward + optimize
    outputs = lstm1(inputs)
    with torch.autocast('cuda'):
      # loss = criterion(outputs, labels, torch.Tensor(outputs.size(0)).cuda().fill_(1.0))
      loss = criterion(outputs, torch.tensor(labels).cuda())
    loss.backward()
    optimizer.step()
    t_loss += loss.item()
    train_loss.append(np.mean(t_loss))

    if epoch % 20 == 0:
      if i % (math.ceil(900*Split/BATCH_SIZE)-1) == 0 and i !=0:
          print(CRED+ f'[{epoch + 1}, {i + 1}] trainning loss: {train_loss[-1]}'+ CRED)
  summary_writer.add_scalar('train_loss', train_loss[-1], epoch)
  print("recorded")

  if epoch % 20 == 0:
    lstm1.eval()
    va_loss = 0
    for i, data in enumerate(test_loader, 0):
      val_x, val_y = data
      val_x, val_y = val_x.to(device), val_y.to(device)
      Testoutput = lstm1(val_x)
      # v_loss = criterion(Testoutput, val_y, torch.Tensor(Testoutput.size(0)).cuda().fill_(1.0))
      v_loss = criterion(Testoutput, val_y) #loss
      va_loss += v_loss.item()
      valid_loss.append(np.mean(va_loss))
      if i % (math.ceil(900*(1-Split)/BATCH_SIZE)-1) == 0 and i !=0:    # print every first
          print(CGREEN+f'[{epoch + 1}, {i + 1}] valid_loss： {valid_loss[-1]}'+CGREEN)
  summary_writer.add_scalar('valid_loss', valid_loss[-1], epoch)
  print("recorded")

  if epoch % 20 == 0:
    GETcorrectnumber(train_loader,CYELLOW)      
    GETcorrectnumber(test_loader,CBLUE)

  lstm1.train()
print('Finished Training')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


[1, 81] trainning loss: 214.1291024684906
recorded
[1, 9] valid_loss： 23.797539234161377
recorded
[1] t accuracy： 8.518518518518519%
[1] t accuracy： 2.2222222222222223%
recorded
recorded
recorded
recorded
recorded
recorded
recorded
recorded
recorded
recorded
recorded
recorded
recorded
recorded
recorded
recorded
recorded
recorded
recorded
recorded
recorded
recorded
recorded
recorded
recorded
recorded
recorded
recorded
recorded
recorded
recorded
recorded
recorded
recorded


KeyboardInterrupt: ignored

In [ ]:
i =1
print(i % 2 == 0 and i !=0)

In [ ]:
# for epoch in range(num_epochs):  # loop over the dataset multiple times
#   t_loss = 0
#   if epoch % 20 == 0:
#     with torch.no_grad():
#       n_correct = 0
#       n_samples = 0
#       accuracy = 0
#       n_class_correct = [0 for i in range(num_classes)]
#       n_class_samples = [0 for i in range(num_classes)]
#       for inputs, labels in train_loader_onehot:
#         inputs = inputs.to(device)
#         labels = labels.to(device)
#         outputs = lstm1(inputs)
#         optimizer.step()          
#         _, predicted = torch.max(outputs, 1)
#         n_samples += labels.size(1)
#         n_correct += (predicted[1] == labels[1]).sum().item()
#         # for i in range(labels.shape[0]):
#         #     label = labels[i]
#         #     pred = predicted[i]
#         #     zeros = torch.Tensor(label.size(0)).cuda().fill_(0.0)
#         #     if (label.equal(pred)):
#         #         for j in range(num_classes):
#         #           if label[j]==1:
#         #             n_class_correct[j] += 1
#         #     n_class_samples[i] += 1
#       acc = 100.0 * n_correct / n_samples
#       print(f'[{epoch + 1}] train accuracy： {acc}%')         

#     with torch.no_grad():
#       n_correct = 0
#       n_samples = 0
#       n_class_correct = [0 for i in range(num_classes)]
#       n_class_samples = [0 for i in range(num_classes)]
#       for inputs, labels in test_loader_onehot:
#         inputs = inputs.to(device)
#         labels = labels.to(device)
#         outputs = lstm1(inputs)
#         optimizer.step()          
#         _, predicted = torch.max(outputs, 1)
#         predicted_onehot = onehot(predicted.shape[0], num_classes, predicted).to(device)
#         for i in range(labels.shape[0]):
#           n_samples += 1
#           if predicted_onehot[i].equal(labels[i]):
#             n_correct += 1
#         print(n_samples, n_correct)
#       acc = 100.0 * n_correct / n_samples
#       print(f'[{epoch + 1}] test accuracy： {acc}%')   

In [ ]:
(predicted_onehot[i].equal(labels[i]))

In [ ]:
labels[1]

In [ ]:
# !rm -rf /logs/ # clear logs
if 'google.colab' in str(get_ipython()): # tensor board
  %load_ext tensorboard  
  # %tensorboard --logdir logs
  %tensorboard --logdir=./models/test

In [ ]:
# import torch.nn.functional as F
# label_test = F.one_hot(labels)

In [ ]:
# label_test[0]

In [ ]:
# from sklearn.preprocessing import StandardScaler, MinMaxScaler
# mm = MinMaxScaler()
# ss = StandardScaler()

# df_X_ss = ss.transform(X_train[:, :-1]) #old transformers
# df_y_mm = mm.transform(y_train[:, -1:]) #old transformers

# df_X_ss = Variable(torch.Tensor(df_X_ss)) #converting to Tensors
# df_y_mm = Variable(torch.Tensor(df_y_mm))
# #reshaping the dataset
# df_X_ss = torch.reshape(df_X_ss, (df_X_ss.shape[0], 1, df_X_ss.shape[1])) 

In [ ]:
# train_predict = lstm1(df_X_ss)#forward pass
# data_predict = train_predict.data.numpy() #numpy conversion
# dataY_plot = df_y_mm.data.numpy()

# data_predict = mm.inverse_transform(data_predict) #reverse transformation
# dataY_plot = mm.inverse_transform(dataY_plot)
# plt.figure(figsize=(10,6)) #plotting
# plt.axvline(x=200, c='r', linestyle='--') #size of the training set

# plt.plot(dataY_plot, label='Actuall Data') #actual plot
# plt.plot(data_predict, label='Predicted Data') #predicted plot
# plt.title('Time-Series Prediction')
# plt.legend()
# plt.show() 

In [ ]:
n_classes = len(np.unique(y_train))
print(n_classes)
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(n_classes)]
    n_class_samples = [0 for i in range(n_classes)]
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        # print(labels.shape)
        outputs = lstm1(inputs)
        optimizer.step()          
        # max returns (value ,index)
        _, predicted = torch.max(outputs, 1)
        # print(predicted)
        n_samples += labels.size(0)
        # print(n_samples)
        n_correct += (predicted == labels).sum().item()
        # print(n_correct)
        # for i in range(24):
        for i in range(n_classes):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[i] += 1
            n_class_samples[i] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')


In [ ]:
predicted

In [ ]:
labels

In [ ]:
label

In [ ]:
len(n_class_samples)

# Mnet

## Load

In [52]:
from torch.autograd import Variable
from torch.cuda import amp
import torch.nn.functional as F
import math
%env CUBLAS_WORKSPACE_CONFIG=:16:8

env: CUBLAS_WORKSPACE_CONFIG=:16:8


In [53]:
Split = 0.90

In [54]:
X_train, y_train = load_MEG_dataset([str(i).zfill(3) for i in range(1,2)], mode = 'concatenate', output_format='numpy',shuffle = False, training=True, train_test_split=Split, batch_size=500)
X_test, y_test = load_MEG_dataset([str(i).zfill(3) for i in range(1,2)], mode = 'concatenate', output_format='numpy',shuffle = False, training=False, train_test_split=Split, batch_size=500)

Loading subject 001
Data loaded
Subject 001 complete
--------------------------------------
Loading subject 001
Data loaded
Subject 001 complete
--------------------------------------


In [55]:
X_train = X_train[:, None, ...]
X_test = X_test[:, None, ...]

In [56]:
X_train.shape

(810, 1, 272, 130)

In [57]:
y_train = (y_train / 2).astype(int) - 1
y_test = (y_test / 2).astype(int) - 1

In [58]:
X_train=np.repeat(X_train,8,axis=3)
X_test=np.repeat(X_test,8,axis=3)

In [59]:
X_train_tensors = torch.Tensor(X_train)
X_test_tensors = torch.Tensor(X_test)
y_train_tensors = torch.from_numpy(y_train.astype(int)) # for cross entropy loss
y_test_tensors = torch.from_numpy(y_test.astype(int))

In [60]:
X_train_tensors.shape

torch.Size([810, 1, 272, 1040])

In [61]:
X_test_tensors.shape

torch.Size([90, 1, 272, 1040])

In [62]:
#reshaping to rows, timestamps, features
# X_train_tensors = torch.permute(X_train_tensors, (3,1,2,0))
# X_test_tensors = torch.permute(X_test_tensors, (3,1,2,0)) 

In [63]:
def onehot(batches, n_classes, y):
  yn = torch.zeros(batches, n_classes)
  for i in range(batches):
    x = [0 for j in range(batches)]
    x[i] = y[i]/2-1                     #ex. [12]-> [5]
    yn[i][int(x[i])]+= 1                  #[000010000]
  return yn

In [64]:
# y_train_tensors_onehot = onehot(int(y_train_tensors.size(0)), len(np.unique(y_train)), y_train_tensors)
# y_test_tensors_onehot = onehot(int(y_test_tensors.size(0)), len(np.unique(y_train)), y_test_tensors)
# print(y_train_tensors_onehot.shape)

In [65]:
y_train_tensors.shape

torch.Size([810])

In [66]:
X_train_tensors=X_train_tensors.cuda()
X_test_tensors=X_test_tensors.cuda()
y_train_tensors=y_train_tensors.cuda()
y_test_tensors=y_test_tensors.cuda()

In [67]:
X_train_tensors.shape

torch.Size([810, 1, 272, 1040])

In [68]:
from scipy.integrate import simps
from mne.time_frequency import psd_array_welch

Test for band power

In [69]:
X_train = np.swapaxes(X_train, 2, -1).squeeze()
data = X_train[100, 20, :]
print(data.shape)

(272,)


In [70]:
psd_mne, freqs_mne = psd_array_welch(data, 250, 1., 70., n_per_seg=int(250/2),
                          n_overlap=int(250/4), n_jobs=1)
for low, high in [(1, 4), (4, 8), (8, 10), (10, 13), (13, 30),
                  (30, 70)]:
    print("processing bands (low, high) : ({},{})".format(low, high))
    # Find intersecting values in frequency vector
    idx_delta = np.logical_and(freqs_mne >= low, freqs_mne <= high)
      # Frequency resolution
    freq_res = freqs_mne[1] - freqs_mne[0]  # = 1 / 4 = 0.25

    # Compute the absolute power by approximating the area under the curve
    power = simps(psd_mne[idx_delta], dx=freq_res)
    print('Absolute power: {:.4f} uV^2'.format(power))
    
    total_power = simps(psd_mne, dx=freq_res)
    rel_power = power / total_power
    
    print('Relative power: {:.4f}'.format(rel_power))

Effective window size : 1.024 (s)
processing bands (low, high) : (1,4)
Absolute power: 0.1126 uV^2
Relative power: 0.2881
processing bands (low, high) : (4,8)
Absolute power: 0.0503 uV^2
Relative power: 0.1287
processing bands (low, high) : (8,10)
Absolute power: 0.0072 uV^2
Relative power: 0.0183
processing bands (low, high) : (10,13)
Absolute power: 0.0077 uV^2
Relative power: 0.0197
processing bands (low, high) : (13,30)
Absolute power: 0.0311 uV^2
Relative power: 0.0797
processing bands (low, high) : (30,70)
Absolute power: 0.1010 uV^2
Relative power: 0.2584


In [71]:
def bandpower_1d(data, sf, band, nperseg=250, relative=False):
    """
        Compute the average power of the signal x in a specific frequency band.
        https://raphaelvallat.com/bandpower.html
    Args:
        data (1d-array):
            Input signal in the time-domain.
        sf (float):
            Sampling frequency of the data.
        band (list):
            Lower and upper frequencies of the band of interest.
        window_sec (float):
            Length of each window in seconds.
            If None, window_sec = (1 / min(band)) * 2
        relative (boolean):
            If True, return the relative power (= divided by the total power of the signal).
            If False (default), return the absolute power.

    Returns:
        bp (float):
            Absolute or relative band power.
    """

    # band = np.asarray(band)
    low, high = band

    # Compute the modified periodogram (Welch)
    psd, freqs = psd_array_welch(data, sf, 1., 70., n_per_seg=int(250/2),
                          n_overlap=int(250/4), n_jobs=1)

    # Frequency resolution
    freq_res = freqs[1] - freqs[0]

    # Find closest indices of band in frequency vector
    idx_band = np.logical_and(freqs >= low, freqs <= high)

    # Integral approximation of the spectrum using Simpson's rule.
    bp = simps(psd[idx_band], dx=freq_res)

    if relative:
        bp /= simps(psd, dx=freq_res)
    return bp

In [72]:
def bandpower(x, fs, fmin, fmax, nperseg=250, relative=True):
    """
    Compute the average power of the multi-channel signal x in a specific frequency band.
    Args:
        x (nd-array): [n_epoch, n_channel, n_times]
           The epoched input data.
        fs (float):
            Sampling frequency of the data.
        fmin (int): Low-band frequency.
        fmax (int): High-band frequency.
        window_sec (float):
            Length of each window in seconds.
            If None, window_sec = (1 / min(band)) * 2
        relative (boolean):
            If True, return the relative power (= divided by the total power of the signal).
            If False (default), return the absolute power.

    Returns:
        bp (nd-array): [n_epoch, n_channel, 1]
            Absolute or relative band power.
    """
    n_epoch, n_channel, _ = x.shape

    bp = np.zeros((n_epoch, n_channel, 1))
    for epoch in range(n_epoch):
        for channel in range(n_channel):
            bp[epoch, channel] = bandpower_1d(
                x[epoch, channel, :],
                fs,
                [fmin, fmax],
                nperseg=nperseg,
                relative=relative,
            )

    return bp

In [73]:
def bandpower_multi(x, fs, bands,  nperseg=250, relative=True):
    """
    Compute the average power of the multi-channel signal x in multiple frequency bands.
    Args:
        x (nd-array): [n_epoch, n_channel, n_times]
           The epoched input data.
        fs (float):
            Sampling frequency of the data.
        bands (list): list of bands to compute the bandpower. echa band is a tuple of fmin and fmax.
        window_sec (float):
            Length of each window in seconds.
            If None, window_sec = (1 / min(band)) * 2
        relative (boolean):
            If True, return the relative power (= divided by the total power of the signal).
            If False (default), return the absolute power.

    Returns:
        bp (nd-array): [n_epoch, n_channel, n_bands]
            Absolute or relative bands power.
    """
    n_epoch, n_channel, _ = x.shape
    bp_list = []
    for idx, band in enumerate(bands):
        fmin, fmax = band
        bp_list.append(
            bandpower(
                x, fs, fmin, fmax,  nperseg=nperseg, relative=relative
            )
        )

    bp = np.concatenate(bp_list, -1)

    return bp

In [ ]:
    bands = [(1, 4), (4, 8), (8, 10), (10, 13), (13, 30), (30, 70)]
    bp = bandpower_multi(
        X_train, fs=100, bands=bands, relative=True
    )

Streaming output truncated to the last 5000 lines.
Effective window size : 2.560 (s)
Effective window size : 2.560 (s)
Effective window size : 2.560 (s)
Effective window size : 2.560 (s)
Effective window size : 2.560 (s)
Effective window size : 2.560 (s)
Effective window size : 2.560 (s)
Effective window size : 2.560 (s)
Effective window size : 2.560 (s)
Effective window size : 2.560 (s)
Effective window size : 2.560 (s)
Effective window size : 2.560 (s)
Effective window size : 2.560 (s)
Effective window size : 2.560 (s)
Effective window size : 2.560 (s)
Effective window size : 2.560 (s)
Effective window size : 2.560 (s)
Effective window size : 2.560 (s)
Effective window size : 2.560 (s)
Effective window size : 2.560 (s)
Effective window size : 2.560 (s)
Effective window size : 2.560 (s)
Effective window size : 2.560 (s)
Effective window size : 2.560 (s)
Effective window size : 2.560 (s)
Effective window size : 2.560 (s)
Effective window size : 2.560 (s)
Effective window size : 2.560 (

## Parms

In [ ]:
CRED    = '\33[31m'
CGREEN  = '\33[32m'
CYELLOW = '\33[33m'
CBLUE   = '\33[34m'

## model

In [ ]:
class ChannelPool(nn.Module):

    def forward(self, x):
        return torch.cat((torch.max(x, 1)[0].unsqueeze(1),
                          torch.mean(x, 1).unsqueeze(1)), dim=1)

In [ ]:
class SpatialAttention(nn.Module):
  def __init__(self):
    super(SpatialAttention, self).__init__()
    self.spatialAttention = nn.Sequential(
        ChannelPool(),
        nn.Conv2d(2, 1, 7, 7, padding=3),
        nn.Sigmoid(),
        )

  def forward(self, x):
    return x * self.spatialAttention(x)


In [ ]:
class Flatten_MEG(nn.Module):
    def forward(self, x):
        return x.view(x.size()[0], -1)

In [ ]:
class ChannelAttention(nn.Module):
    """
            Implementation of a channel attention module.
        """

    def __init__(self, shape, reduction_factor=16):

        super(ChannelAttention, self).__init__()

        _, in_channel, h, w = shape

        self.mlp = nn.Sequential(
            Flatten_MEG(),
            nn.Linear(in_channel, in_channel // reduction_factor),
            nn.ReLU(),
            nn.Linear(in_channel // reduction_factor, in_channel),
        )
        self.avg = nn.AvgPool2d(kernel_size=(h, w), stride=(h, w))
        self.max = nn.MaxPool2d(kernel_size=(h, w), stride=(h, w))

    def forward(self, x):

        avg = self.avg(x)
        max = self.max(x)

        attention = (
            torch.sigmoid(self.mlp(avg) + self.mlp(max))
            .unsqueeze(2)
            .unsqueeze(3)
        )

        return x * attention


In [ ]:
class MNet(nn.Module):
    """
        Model inspired by [Aoe at al., 10.1038/s41598-019-41500-x]
    """

    def __init__(self, n_times):
        """
        Args:
            n_times (int):
                n_times dimension of the input data.
        """
        super(MNet, self).__init__()
        self.n_times = n_times
        # if n_times == 501:  # TODO automatic n_times
        #     self.n_times = 12
        # elif n_times == 601:
        #     self.n_times = 2
        # elif n_times == 701:
        #     self.n_times = 4
        # else:
        #     raise ValueError("Network can work only with n_times = 501, 601, "
        #                      "701 (epoch duration of 1., 1.2, 1.4 sec),"
        #                      " got instead {}".format(n_times))

        self.spatial = nn.Sequential(
            nn.Conv2d(1, 32, stride=(1,2), kernel_size=(272,64), bias=False), #kernel size 204, 64
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Conv2d(32, 64, kernel_size=(1, 16), bias=False), # kernel size 1,16
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(1, 2), stride=(1, 2)),
            nn.BatchNorm2d(64),
        )

        self.temporal = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=(8, 8), bias=False),
            nn.ReLU(),
            # nn.BatchNorm2d(32),
            nn.Conv2d(32, 32, kernel_size=(8, 8), bias=False),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(1, 3), stride=(1, 2)),
            # nn.BatchNorm2d(32),
            nn.Conv2d(32, 64, kernel_size=(6, 6), bias=False),
            nn.ReLU(),
            # nn.BatchNorm2d(64),
            nn.Conv2d(64, 64, kernel_size=(6, 6), bias=False),
            nn.ReLU(),
            # nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size=(1, 2), stride=(1, 2)),
            nn.Conv2d(64, 128, kernel_size=(5, 5), bias=False),
            nn.ReLU(),
            nn.Dropout2d(p=0.3),
            # nn.BatchNorm2d(128),
            nn.Conv2d(128, 128, kernel_size=(5, 5), bias=False),
            nn.ReLU(),
            nn.Dropout2d(p=0.3),
            # nn.BatchNorm2d(128),
            nn.MaxPool2d(kernel_size=(1, 2), stride=(1, 2)),
            nn.Conv2d(128, 256, kernel_size=(4, 4), bias=False),
            nn.ReLU(),
            nn.Dropout2d(p=0.3),
            # nn.BatchNorm2d(256),
            nn.Conv2d(256, 256, kernel_size=(4, 4), bias=False),
            nn.ReLU(),
            nn.Dropout2d(p=0.3),
            # nn.BatchNorm2d(256),
        )

        self.attention = nn.Sequential(
            ChannelAttention([None, 256, 26, self.n_times]), SpatialAttention()
        )

        self.flatten = Flatten_MEG()

        self.ff = nn.Sequential(
            nn.Linear(256 * 26 * self.n_times, 1024),
            nn.BatchNorm1d(num_features=1024),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(1024, 1024),
            nn.BatchNorm1d(num_features=1024),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(1024, 14),
        )

    def forward(self, x):
        x = self.spatial(x)
        print(x.shape)
        x = torch.transpose(x, 1, 2)
        print(x.shape)        
        x = self.temporal(x)
        # x = self.attention(x)
        x = self.ff(self.flatten(x))

        return x.squeeze(1)

In [ ]:
mnet = MNet(11).cuda()
print(mnet)

MNet(
  (spatial): Sequential(
    (0): Conv2d(1, 32, kernel_size=(272, 64), stride=(1, 2), bias=False)
    (1): ReLU()
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv2d(32, 64, kernel_size=(1, 16), stride=(1, 1), bias=False)
    (4): ReLU()
    (5): MaxPool2d(kernel_size=(1, 2), stride=(1, 2), padding=0, dilation=1, ceil_mode=False)
    (6): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (temporal): Sequential(
    (0): Conv2d(1, 32, kernel_size=(8, 8), stride=(1, 1), bias=False)
    (1): ReLU()
    (2): Conv2d(32, 32, kernel_size=(8, 8), stride=(1, 1), bias=False)
    (3): ReLU()
    (4): MaxPool2d(kernel_size=(1, 3), stride=(1, 2), padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(32, 64, kernel_size=(6, 6), stride=(1, 1), bias=False)
    (6): ReLU()
    (7): Conv2d(64, 64, kernel_size=(6, 6), stride=(1, 1), bias=False)
    (8): ReLU()
    (9): MaxPool2d(kernel_size=(1, 2), stride=(1,

## original Aoe model

In [ ]:
class Concatenate(nn.Module):
    def __init__(self):
        super(Concatenate, self).__init__()

    def forward(self, x, bp):

        # min_ = x.min(1, keepdim=True)[0]
        # if min_[0] < 0:
        #     x = x + min_
        # else:
        #     x = x - min_
        # x = x / x.max()
        x = x.view(x.shape[0], -1)
        bp = bp.view(bp.shape[0], -1)
        x = torch.cat([x, bp], -1)

        return x

In [ ]:
class AoeMNet(nn.Module):
    """
        Model inspired by [Aoe at al., 10.1038/s41598-019-41500-x]
    """
    def __init__(self, n_times):
        super(AoeMNet, self).__init__()
        self.n_times = n_times
        self.spatial = nn.Sequential(
            nn.Conv2d(1, 32, stride=(1,2), kernel_size=(272,64), bias=False), #kernel size 204, 64
            nn.ReLU(),
            # nn.BatchNorm2d(32),
            nn.Conv2d(32, 64, stride=(1,2), kernel_size=(1, 16), bias=False), # kernel size 1,16
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(1, 2), stride=(1, 2)),
            # nn.BatchNorm2d(64),
        )

        self.temporal = nn.Sequential(
            nn.Conv2d(1, 32, stride=(1, 1), kernel_size=(8, 8), bias=False),
            nn.ReLU(),
            # nn.BatchNorm2d(32),
            nn.Conv2d(32, 32, stride=(1, 1), kernel_size=(8, 8), bias=False),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(5, 3), stride=(5, 3)),
            # nn.BatchNorm2d(32),
            nn.Conv2d(32, 64, stride=(1, 1), kernel_size=(1, 4), bias=False),
            nn.ReLU(),
            # nn.BatchNorm2d(64),
            nn.Conv2d(64, 64, stride=(1, 1), kernel_size=(1, 4), bias=False), #conv6
            nn.ReLU(),
            # nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size=(1, 2), stride=(1, 2)),
            nn.Conv2d(64, 128, stride=(1, 1), kernel_size=(1, 2), bias=False),
            nn.ReLU(),
            # nn.Dropout2d(p=0.3),
            # nn.BatchNorm2d(128),
            nn.Conv2d(128, 128, stride=(1, 1), kernel_size=(1, 2), bias=False),
            nn.ReLU(),
            # nn.Dropout2d(p=0.3),
            # nn.BatchNorm2d(128),
            nn.MaxPool2d(kernel_size=(1, 2), stride=(1, 2)),
            nn.Conv2d(128, 256, stride=(1, 1), kernel_size=(1, 2), bias=False),
            nn.ReLU(),
            # nn.Dropout2d(p=0.3),
            # nn.BatchNorm2d(256),
            nn.Conv2d(256, 256, stride=(1, 1), kernel_size=(1, 2), bias=False), #conv10
            nn.ReLU(),
            # nn.Dropout2d(p=0.3),
            # nn.BatchNorm2d(256),
            nn.MaxPool2d(kernel_size=(1, 2), stride=(1, 2)),

        )

        self.attention = nn.Sequential(
            ChannelAttention([None, 256, 26, self.n_times]), SpatialAttention()
        )

        self.flatten = Flatten_MEG()
        # self.concatenate = Concatenate()

        self.ff = nn.Sequential(
            nn.Linear(64 * 2 * self.n_times + 272 * 6, 1024),
            # nn.Linear(5120,1024),
            # nn.BatchNorm1d(num_features=1024),
            nn.ReLU(),
            # nn.Dropout(0.3),
            nn.Linear(1024, 1024),
            # nn.BatchNorm1d(num_features=1024),
            nn.ReLU(),
            # nn.Dropout(0.3),
            nn.Linear(1024, 14),
        )
    def forward(self, x):
        x = self.spatial(x)
        # print(x.shape)
        x = torch.transpose(x, 1, 2)
        # print(x.shape)        
        x = self.temporal(x)
        x = self.attention(x)
        # print(x.shape)
        x = self.flatten(x)
        # print(x.shape)
        x = self.ff(x)

        return x.squeeze(1)
    # def forward(self, x, pb):
    #     x = self.spatial(x)
    #     x = torch.transpose(x, 1, 2)
    #     x = self.temporal(x)
    #     x = self.concatenate(x)
    #     x = self.ff(self.flatten(x))

    #     return x.squeeze(1)

In [ ]:
aoemnet = AoeMNet(11).cuda()
print(aoemnet)

AoeMNet(
  (spatial): Sequential(
    (0): Conv2d(1, 32, kernel_size=(272, 64), stride=(1, 2), bias=False)
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(1, 16), stride=(1, 2), bias=False)
    (3): ReLU()
    (4): MaxPool2d(kernel_size=(1, 2), stride=(1, 2), padding=0, dilation=1, ceil_mode=False)
  )
  (temporal): Sequential(
    (0): Conv2d(1, 32, kernel_size=(8, 8), stride=(1, 1), bias=False)
    (1): ReLU()
    (2): Conv2d(32, 32, kernel_size=(8, 8), stride=(1, 1), bias=False)
    (3): ReLU()
    (4): MaxPool2d(kernel_size=(5, 3), stride=(5, 3), padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(32, 64, kernel_size=(1, 4), stride=(1, 1), bias=False)
    (6): ReLU()
    (7): Conv2d(64, 64, kernel_size=(1, 4), stride=(1, 1), bias=False)
    (8): ReLU()
    (9): MaxPool2d(kernel_size=(1, 2), stride=(1, 2), padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(64, 128, kernel_size=(1, 2), stride=(1, 1), bias=False)
    (11): ReLU()
    (12): Conv2d(128, 128, kernel_size=(

In [ ]:
def GETcorrectnumber(loader, printcolor):
  with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(num_classes)]
    n_class_samples = [0 for i in range(num_classes)]
    for inputs, labels in loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = aoemnet(inputs)
        optimizer.step()          
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        # n_correct += (predicted == labels).sum().item()
        for k in range(predicted.shape[0]):
          if predicted[k]==labels[k]:
            n_correct +=1
        # for i in range(num_classes): # accuracy for each class
        #     label = labels[i]
        #     pred = predicted[i]
        #     if (label == pred):
        #         n_class_correct[i] += 1
        #     n_class_samples[i] += 1
    acc = 100.0 * n_correct / n_samples
    print(printcolor+f'[{epoch + 1}] t accuracy： {acc}%'+printcolor)
  return acc

## Train

In [ ]:
BATCH_SIZE=32
train = Data.TensorDataset(X_train_tensors, y_train_tensors)
test = Data.TensorDataset(X_test_tensors, y_test_tensors)
train_loader = Data.DataLoader(train, batch_size = BATCH_SIZE, shuffle = False)
test_loader = Data.DataLoader(test, batch_size = BATCH_SIZE, shuffle = False)

learning_rate = 0.0001
criterion = torch.nn.CrossEntropyLoss()
# criterion = torch.nn.MSELoss()
# optimizer = torch.optim.Adam(mnet.parameters(), lr=learning_rate) 
optimizer = torch.optim.SGD(mnet.parameters(), lr=learning_rate) 

In [ ]:
from torch.utils.tensorboard import SummaryWriter
summary_writer = SummaryWriter(f'./models/test')
train_loss = []
valid_loss = []
aoemnet.train()
for epoch in range(num_epochs):  # loop over the dataset multiple times
  t_loss = 0
  for i, data in enumerate(train_loader, 0):
    # get the inputs; data is a list of [inputs, labels]
    inputs, labels = data
    # print(inputs.shape)
    inputs, labels = inputs.to(device), labels.to(device)
    # zero the parameter gradients
    optimizer.zero_grad()
    # forward + backward + optimize
    outputs = aoemnet(inputs)
    # with torch.autocast('cuda'):
    # loss = criterion(outputs, torch.tensor(labels).cuda())
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    t_loss += loss.item()
    # print(t_loss)
    train_loss.append(np.mean(t_loss))

    if epoch % 1 == 0:
      if i % (math.ceil(900*Split/BATCH_SIZE)-1) == 0 and i !=0:
          print(CRED+ f'[{epoch + 1}, {i + 1}] trainning loss: {train_loss[-1]}'+ CRED)
  summary_writer.add_scalar('train_loss', train_loss[-1], epoch)

  if epoch % 1 == 0:
    aoemnet.eval()
    va_loss = 0
    for i, data in enumerate(test_loader, 0):
      val_x, val_y = data
      val_x, val_y = val_x.to(device), val_y.to(device)
      Testoutput = aoemnet(val_x)
      # v_loss = criterion(Testoutput, val_y, torch.Tensor(Testoutput.size(0)).cuda().fill_(1.0))
      v_loss = criterion(Testoutput, val_y) #loss
      va_loss += v_loss.item()
      valid_loss.append(np.mean(va_loss))
      if i % (math.ceil(900*(1-Split)/BATCH_SIZE)-1) == 0 and i !=0:    # print every first
          print(CGREEN+f'[{epoch + 1}, {i + 1}] valid_loss： {valid_loss[-1]}'+CGREEN)
  summary_writer.add_scalar('valid_loss', valid_loss[-1], epoch)

  # if epoch % 1 == 0:
  #   GETcorrectnumber(train_loader,CYELLOW)      
  #   GETcorrectnumber(test_loader,CBLUE)

  # aoemnet.train()
print('Finished Training')

RuntimeError: ignored

Confusion matrix and mean accuracy

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, ConfusionMatrixDisplay
confusion_matrix_test_loader = Data.DataLoader(test, batch_size = 90, shuffle = True)
with torch.no_grad():
    list_mean_accuracy = []
    for inputs, labels in confusion_matrix_test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = aoemnet(inputs)
        optimizer.step()          
        _, predicted = torch.max(outputs, 1)
        labels = labels.cpu()
        predicted = predicted.cpu()
        labels = labels.numpy()
        predicted = predicted.numpy()
        mean_conf_mat = confusion_matrix(labels, predicted)
        mean_accuracy = accuracy_score(labels[labels != 99], predicted[predicted != 99])
        mean_conf_mat = mean_conf_mat.astype('float') / mean_conf_mat.sum(axis=1)  # normalise
        list_mean_accuracy.append(mean_accuracy)
        print("Mean accuracy = {0}".format(mean_accuracy))
        ConfusionMatrixDisplay.from_predictions(labels, predicted)
        # plt.savefig('/content/drive/MyDrive/MT_ML_Decoding/Aversive_state_reactivation/notebooks/templates/save_folder/fig-{}.png'.format(session_id), dpi=600)
        plt.show()

In [ ]:
# !rm -rf /logs/ # clear logs
if 'google.colab' in str(get_ipython()): # tensor board
  %load_ext tensorboard  
  # %tensorboard --logdir logs
  %tensorboard --logdir=./models/test